# Prophet

In [ ]:
# import packages

import pandas as pd
import numpy as np
from fbprophet import Prophet

In [ ]:
Prophet?

## Steel

In [ ]:
# import steel production

df_prod = pd.read_csv('../data/raw/Industry/SteelHistorical_datetime.csv')
df_prod.head()

In [ ]:
df_prod.tail()

In [ ]:
df_macro = pd.read_csv('../data/raw/Industry/MacroHistorical_datetime.csv')

In [ ]:
df_macro.head()

In [ ]:
df_macro.tail()

In [ ]:
# create per capita

df_macro['GDP_per_capita'] = df_macro['GDP'].div(df_macro['Population'])

In [ ]:
# merge dataframes

df = pd.merge(df_prod,df_macro,how='left',on=['Economy','Year'])

In [ ]:
# Select AUS
df = df.set_index('Economy')
df = df.loc['PRC']

In [ ]:
# make Year ds and use only AUS
df =df.reset_index()
df.head()

In [ ]:
df.tail()

In [ ]:
# create natural log of production per capita as a feature

df['ln_prod_per_cap'] = df['SteelConsumption'].div(df['Population'])
df['ln_prod_per_cap'] = np.log(df['ln_prod_per_cap'])

df['ln_GDP_per_cap'] = np.log(df['GDP_per_capita'])

df.head()

In [ ]:
df = df.rename(columns={"Economy": "e", "Year": "ds","ln_prod_per_cap":"y"})
df.head()

In [ ]:
m = Prophet(daily_seasonality=False,
            weekly_seasonality=False,
            yearly_seasonality=False,
            seasonality_mode='multiplicative',
            growth='linear')

m.add_regressor('ln_GDP_per_cap')
m.fit(df)

In [ ]:
# add future GDP

df_future_macro = pd.read_csv('../data/raw/Industry/MacroAssumptions_datetime.csv')
df_future_macro.head()

In [ ]:
# create per capita

df_future_macro['GDP_per_capita'] = df_future_macro['GDP'].div(df_future_macro['Population'])

In [ ]:
# select only AUS

df_future_macro = df_future_macro.set_index('Economy')
df_future_macro = df_future_macro.loc['PRC']
df_future_macro = df_future_macro.reset_index()
df_future_macro.tail()

In [ ]:
# create natural log of GDP per capita
df_future_macro['ln_GDP_per_cap'] = np.log(df_future_macro['GDP_per_capita'])
df_future_macro.head()

In [ ]:
# extract GDP_per_capita for 1980-2050

df1 = pd.DataFrame(columns = ['ln_GDP_per_cap'])
df2 = pd.DataFrame(columns = ['ln_GDP_per_cap'])

df1['ln_GDP_per_cap'] = df['ln_GDP_per_cap']
df2['ln_GDP_per_cap'] = df_future_macro['ln_GDP_per_cap']
#df1 = df['GDP_per_capita']
#df2 = df_future_macro['GDP_per_capita']

In [ ]:
df2.info()

In [ ]:
df_future = pd.concat([df1,df2],
                     ignore_index=True)

In [ ]:
df_future.info()

In [ ]:
m.make_future_dataframe?

In [ ]:
future = m.make_future_dataframe(periods=34,
                                 freq = 'Y',
                                 include_history=True)
future.head()

In [ ]:
future.tail()

In [ ]:
future['ln_GDP_per_cap']=df_future['ln_GDP_per_cap']

In [ ]:
future.head()

In [ ]:
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)